In [1]:
# Instalar dependencias en Colab
!apt-get install -y tesseract-ocr
!pip install pymupdf pytesseract sentence-transformers faiss-cpu googletrans==4.0.0-rc1
!pip install fitz
!pip install tools
!pip install --upgrade pymupdf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 21 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (4,927 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [2]:
!pip install --upgrade pymupdf

In [3]:
import pymupdf  # PyMuPDF
import pytesseract
import cv2
import numpy as np
from PIL import Image
import re
from sentence_transformers import SentenceTransformer
import faiss
from google.colab import files
import tempfile
from googletrans import Translator

In [4]:
# Cargar modelo de embeddings
model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
translator = Translator()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
def extract_text_from_pdf(pdf_path):
    doc = pymupdf.open(pdf_path)
    text = ""
    images = []

    for page in doc:
        page_text = page.get_text("text")
        page_text = page_text.replace("/2018", "")
        page_text = page_text.replace("Seguro 2030: el impacto de la IA en el futuro del seguro |McKinsey & Company", "")
        page_text = page_text.replace("https://www.mckinsey.com/industries/financial-services/our-insights/insurance-2030-the-impact-of-ai-on-the-future-of-insurance", "")
        page_text = re.sub(r'\b\d+/\d+\b', "", page_text)  # Eliminar cadenas como 1/13 hasta 13/13
        text += page_text + "\n"

        for img in page.get_images(full=True):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image = Image.open(base_image["image"])
            images.append(image)
            text += pytesseract.image_to_string(image) + "\n"

    return text, images

def analyze_pie_chart(image):
    image_np = np.array(image)
    gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100, param1=50, param2=30, minRadius=50, maxRadius=200)
    if circles is not None:
        return "El documento contiene un gráfico de tipo circular (pie chart)."
    return ""

def create_vector_store(text):
    sentences = list(set([s.strip() for s in text.split(".\n") if len(s.strip()) > 10]))  # Eliminar duplicados
    embeddings = model.encode(sentences, convert_to_numpy=True, normalize_embeddings=True)
    index = faiss.IndexFlatIP(embeddings.shape[1])  # Distancia por producto interno para más precisión
    index.add(embeddings)
    return index, sentences

def search_document(query, index, sentences):
    query_embedding = model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D, I = index.search(query_embedding, k=3)  # Buscar los 3 más relevantes
    results = list(set([sentences[i] for i in I[0] if i < len(sentences)]))  # Evitar repeticiones
    return "\n".join(results) if results else "No encontré información relevante en el documento."

def translate_to_spanish(text):
    return translator.translate(text, dest='es').text

In [6]:
# Subir archivo en Colab
uploaded = files.upload()
for filename, file_data in uploaded.items():
    with open(filename, "wb") as f:
        f.write(file_data)
    pdf_path = filename

    text, images = extract_text_from_pdf(pdf_path)
    index, sentences = create_vector_store(text)

    # Analizar imágenes si existen
    additional_info = ""
    for img in images:
        additional_info += analyze_pie_chart(img) + "\n"

    print("Chat activado. Escribe 'salir' para terminar.")
    while True:
        query = input("Haz una pregunta sobre el documento: ")
        if query.lower() == "salir":
            print("Saliendo del chat.")
            break
        response = search_document(query, index, sentences)
        translated_response = translate_to_spanish(response)
        print(translated_response)
        print("**********************************************")
    if additional_info.strip():
        print("Información adicional detectada en el documento:")
        print(translate_to_spanish(additional_info))

Saving 1210-Insurance-2030-The-impact-of-AI-on-the-future-of-insurance-_-McKinsey-Company (1) (1).pdf to 1210-Insurance-2030-The-impact-of-AI-on-the-future-of-insurance-_-McKinsey-Company (1) (1).pdf
Chat activado. Escribe 'salir' para terminar.
Haz una pregunta sobre el documento: Advances in cognitive technologies
2. Cambio profundo: puntos de inflexión tecnológica e impacto social, Foro Económico Mundial,
Septiembre de 2015, worforum.org
Avances en tecnologías cognitivas
Redes neuronales convolucionales y otras tecnologías de aprendizaje profundo que se utilizan actualmente
Principalmente para el procesamiento de imagen, voz y texto no estructurado evolucionará para aplicarse
en una amplia variedad de aplicaciones.Estas tecnologías cognitivas, que se basan libremente
Sobre la capacidad del cerebro humano para aprender a través de la descomposición y la inferencia,
convertirse en el enfoque estándar para procesar los datos increíblemente grandes y complejos
flujos que serán generados